# Avaliação de sentimentos sobre filmes usando API do Reddit

Neste projeto usamos técnicas de Tecnologia e Análise de Comportamentos para analisar reviews de filmes no subreddit "TrueFilm". 

As reviews estão classificadas como "Negative", "Neutral", "Positive" dando nos assim oportunidade de melhor análise dos sentimentos mais comuns sobre alguns filmes. 

Nesta subreddit existem muitas opiniões variadas sobre diversos filmes por isso decidimos retirar só os 250 comentários com mais interações do ultimo ano.   

Usamos também a ferramenta PowerBI para realizar um dashboard que nos dá a oportunidade de ter uma melhor visão sobre o que acontece no nosso dataset, podendo assim tirar melhores conclusões e poder ver a informação de forma organizada.

O objetivo deste trabalho é melhorar as nossas competencias na area de Tecnologia e Análise de Comportamentos e na comunicação dos resultados.
 

Importamos a biblioteca praw, que nos dá acesso à API do Reddit 

In [ ]:
import praw

Para termos acesso à API necessitamos de autenticar a nossa conta

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

CLIENT_ID = os.getenv("R_CLIENT_ID")
CLIENT_SECRET = os.getenv("R_CLIENT_SECRET")
USER_AGENT = os.getenv("R_USER_AGENT")
USERNAME = os.getenv("R_USERNAME")
PASSWORD = os.getenv("R_PASSWORD")

In [ ]:
reddit_instance = praw.Reddit(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    user_agent=USER_AGENT,
    username=USERNAME,
    password=PASSWORD
)

Usamos a função *subreddit('TrueFilm')* para entrarmos na subreddit que nos interessa

In [ ]:
subreddit = reddit_instance.subreddit('TrueFilm')
subreddit

#### Com os seguintes codigos podemos observar algumas informações sobre o subreddit

In [ ]:
print(subreddit.title)

In [ ]:
print(subreddit.description)

In [ ]:
print(subreddit.subscribers)

Para acessar os comentários com mais interações, utilizamos a função *top* com os parâmetros *limit* e *time_filter*, que respetivamente restringem o número de posts e definem o período de tempo considerado.

Criamos também um ciclo for para acessar as todos os 250 comentários retirando o titulo, conteudo e data de publicação e alteramos a data para o formato YYYY-MM-DD HH:MM:SS e guardamos tudo num Excel chamado *reddit_top_posts*.

In [ ]:
import pandas as pd
from datetime import datetime

top_posts = subreddit.top(limit=250, time_filter="year")

posts_data = []
for post in top_posts:
    # Converte o timestamp Unix para um formato de data legível
    post_date = datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S')
    
    # Adiciona os dados do post à lista
    posts_data.append({
        'Title': post.title,
        'Content': post.selftext,
        'Created': post_date  # Data de criação
    })
df = pd.DataFrame(posts_data)
df.to_excel('reddit_top_posts.xlsx', index=False)

## Pré-Processamento

Primeiro, para conseguirmos extrair mais informação retiramos as stopwords dos comentários.

Depois utilizamos o código para analisar o sentimento dos comentários retirados do Excel *reddit_top_posts*. Para isso, carregamos um modelo de classificação de sentimento pré-treinado (RoBERTa) e seu tokenizer, ambos disponíveis através da biblioteca `transformers`.

Foi criada uma função chamada *analyze_sentiment* que recebe o conteúdo do post, tokeniza o texto e o passa pelo modelo para gerar uma previsão de sentimento. O modelo retorna as probabilidades de cada classe (negativo, neutro, positivo) e o sentimento mais provável é selecionado. 

Em seguida, aplicamos essa função ao conteúdo dos posts, criando uma nova coluna no DataFrame chamada *sentiment*. Por fim, o DataFrame atualizado, contendo as previsões de sentimento, é salvo num novo arquivo Excel chamado *reddit_top_posts_with_sentiment*.

In [ ]:
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Carrega o dataset dos posts que criaste
df = pd.read_excel('reddit_top_posts.xlsx')
stop_words = set(stopwords.words('english'))

# Função para remover stopwords
def remove_stopwords(text):
    words = word_tokenize(text)  # Tokenizar o texto em palavras
    words_filtered = [word for word in words if word.lower() not in stop_words]
    return ' '.join(words_filtered)

# Aplicar a função na coluna de textos
df['Content'] = df['Content'].apply(remove_stopwords)

# Carrega o modelo e o tokenizer RoBERTa
model = AutoModelForSequenceClassification.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment')
tokenizer = AutoTokenizer.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment')

# Função para analisar o sentimento do texto com tokenização manual
def analyze_sentiment(text):
    if isinstance(text, str) and pd.notnull(text):
        # Tokeniza o texto
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
        
        # Passa pelo modelo
        with torch.no_grad():
            outputs = model(**inputs)
        
        # Obtém as probabilidades e determina o rótulo de sentimento
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        sentiment_label = torch.argmax(probs).item()
        
        # Mapeia os índices para os rótulos de sentimento: 0=Negative, 1=Neutral, 2=Positive
        labels = ['Negative', 'Neutral', 'Positive']
        return labels[sentiment_label]
    else:
        return "No analysis"

# Aplica a função de análise de sentimento ao conteúdo do post e cria uma nova coluna 'sentiment'
df['sentiment'] = df['Content'].apply(analyze_sentiment)

# Salva o DataFrame com a nova coluna de sentimento em um novo arquivo Excel
df.to_excel('reddit_top_posts_with_sentiment.xlsx', index=False)


# Dashboard

<img src="power_bi.png" height="1000">

## Conclusão

A ferramenta utilizada para o desenvolvimento do dashboard foi o Power BI, escolhida por já termos experiência prévia com ela em outra cadeira, o que facilitou o processo. O objetivo principal foi realizar uma Análise de Sentimentos das Reviews de Filmes. Criamos um gráfico de barras que apresenta a quantidade de comentários classificados como neutros, positivos e negativos. No entanto, como os números exatos não eram facilmente visíveis no gráfico, adicionamos uma "caixa de cartões", que exibe o total de comentários.
Para aprofundar a análise, incluímos três filtros interativos: o primeiro, de ano e mês, para identificar os períodos com maior volume de comentários, o segundo, de dias da semana, para descobrir em quais dias as pessoas mais comentam ou interagem e o terceiro, de título e texto do comentário, para explorar detalhes específicos e identificar rapidamente sentimentos positivos ou negativos. Além disso, criamos um segundo gráfico de barras, que mostra os dias e meses com maior volume de comentários, permitindo uma visão mais clara das tendências temporais.
Para complementar o dashboard, adicionamos uma "Word Cloud" (Nuvem de Palavras), que destacou as palavras mais frequentes nos comentários, como nomes de filmes ou adjetivos comuns. Também implementamos uma barra de pesquisa, que possibilitou a busca de qualquer palavra, frase ou nome de filme, mostrando se os comentários que contêm essa palavra foram classificados como positivos, neutros ou negativos, bem como os dias e anos em que apareceram e outras palavras associadas.
Com base no dataset analisado, concluímos que o mês com maior volume de comentários foi março, o dia da semana mais comentado foi a quarta-feira, e a maioria dos comentários apresenta um sentimento neutro. Em resumo, o dashboard possibilitou uma análise clara e interativa, identificando tendências de comportamento nos comentários e oferecendo insights úteis sobre a receção de filmes em diferentes períodos e contextos. Essas conclusões reforçam o potencial do Power BI para transformar dados complexos em informações acessíveis e significativas.